# Overview (From README)

A TensorFlow implementation of a Temporal Convolutional Network (TCN) to classify individuals based on their gait patterns in the Har-Lab Dataset.  
**Not activity-specific:** gait patterns are tied to individuals across activities.
  
**Performance Evaluation**  
(10 ms per timestep)
* `timesteps=100`: 92% test accuracy
* `timesteps=200`: 96% test accuracy
* `timesteps=300`: 98% test accuracy
  
This is actually really interesting, as gait cycles are typically 1-1.5 seconds, potentially explaining the significant accuracy increase between `timesteps=100` and `timesteps=200` (6%), as opposed to the mild increase between `timesteps=200` and `timesteps=300` (2%).
  
**Credibility**: since data was collected in contiguous sequences per activity/person, it's difficult to define a set for testing. As a reasonable approach, the last 20% of each activity sequence per person (walk_mixed, walk_sidewalk, walk_treadmill), is reserved in a contiguous block to evaluate the model. This approach shows reasonable proof of generalizing and learning as opposed to that of randomly splitting sequences into train and test, which reaches an unlikely 99% test accuracy after 3 epochs with `timesteps=100`.  

**Required packages:** pandas, numpy, tensorflow, keras

# Define Constants

In [ ]:
# ====== Data ======
timesteps = 300 # Sequence length: 10 ms per timestep
activity_types = ["walk_mixed", "walk_treadmill", "walk_sidewalk"]
test_fraction = 0.2 # Percentage of data reserved for testing
batch_size = 128 
seed = 42

# ====== Model ======
num_filters = 64 # Number of filters in the TCN
kernel_size = 3 # Convolutional kernel size in the TCN
num_blocks = 3 # Number of convolutional blocks in the TCN
dropout_rate = 0.25 # Percentage of neurons to drop in the TCN

# ====== Training ======
learning_rate = 1e-3
num_epochs = 20

# Load Dataset

In [ ]:
# Parse csv data into numpy arrays
from data_utils import build_temporal_dataset

# Train on mixed walking
X_train, y_train, X_test, y_test = build_temporal_dataset(
                                        directory_path="dataset", 
                                        temporal_dim=timesteps, 
                                        test_fraction=test_fraction,
                                        filter_files_by_column_count=15, # Filters out files missing either ankle, waist, or wrist data
                                        activity_types=activity_types,
                                   )

# Automatically infer constants
num_classes = len(set(y_train))
num_features = len(X_train[0][0])

Skipping dataset/002_labeled.csv: column count 16 != 15
Skipping dataset/026_labeled.csv: column count 11 != 15
Skipping dataset/027_labeled.csv: column count 11 != 15
Skipping dataset/028_labeled.csv: column count 11 != 15
Skipping dataset/029_labeled.csv: column count 11 != 15
Skipping dataset/030_labeled.csv: column count 11 != 15
Skipping dataset/040_labeled.csv: column count 16 != 15
Train: 3665 sequences, Test: 957 sequences


In [ ]:
# Convert to TensorFlow datasets
import tensorflow as tf

tf.random.set_seed(seed)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
    .shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)) \
    .batch(batch_size).prefetch(tf.data.AUTOTUNE)


2025-09-07 01:00:17.171124: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Max
2025-09-07 01:00:17.171153: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-09-07 01:00:17.171158: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2025-09-07 01:00:17.171173: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-07 01:00:17.171182: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Build Model

In [5]:
from model_utils import build_model

model = build_model(timesteps=timesteps, num_features=num_features, num_classes=num_classes, num_filters=num_filters, kernel_size=kernel_size, dropout_rate=dropout_rate, num_blocks=num_blocks)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 12)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 300, 12)   │         48 │ input_layer[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 300, 12)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 300, 64)   │      2,368 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 300, 64)   │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 300, 64)   │        256 │ dropout[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 300, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 300, 64)   │     12,352 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 300, 64)   │        832 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 300, 64)   │          0 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 300, 64)   │          0 │ conv1d_2[0][0],   │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 300, 64)   │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 300, 64)   │        256 │ activation_2[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 300, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 300, 64)   │     12,352 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 300, 64)   │          0 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 300, 64)   │        256 │ dropout_2[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 300, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 300, 64)   │     12,352 │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 78,091 (305.04 KB)

 Trainable params: 77,427 (302.45 KB)

 Non-trainable params: 664 (2.59 KB)

# Train Model

In [6]:
# Compile the model with Adam
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    # Sparse crossentropy since labels are integers, not one-hot
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=num_epochs,
)

Epoch 1/20


2025-09-07 01:00:18.013413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 93ms/step - accuracy: 0.1080 - loss: 3.1927
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.2467 - loss: 2.5985
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.4106 - loss: 1.8463
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.5880 - loss: 1.2868
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.7165 - loss: 0.8514
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.7804 - loss: 0.6577
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.8619 - loss: 0.4583
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.8712 - loss: 0.3999
Epoch 9/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9031 - loss: 0.3097
Epoch 10/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.9302 - loss: 0.2412
Epoch 11/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9473 - loss: 0.1782
Epoch 12/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.9550 - l

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9843 - loss: 0.0407  
Test Accuracy: 0.9843, Test Loss: 0.0407


In [9]:
# Save the model
model.save_weights(f"saved_models/har-lab_tcn_timesteps:{timesteps}_accuracy:{(test_acc * 100):.2f}%.weights.h5")